# Sistemas expertos usando la libreria experta


## Primer ejemplo

In [16]:
! pip install experta

In [17]:
import collections.abc
if not hasattr(collections, 'Mapping'):
    collections.Mapping = collections.abc.Mapping

In [18]:

from experta import Fact, Rule, KnowledgeEngine

# Definimos los hechos
class Sintoma(Fact):
    pass

# Creamos el sistema experto
class DiagnosticoMedico(KnowledgeEngine):

    @Rule(Sintoma(nombre="fiebre"))
    def enfermedad_fiebre(self):
        print("Podría tener una infección.")

    @Rule(Sintoma(nombre="dolor de cabeza"))
    def enfermedad_cabeza(self):
        print("Podría ser migraña o estrés.")

# Instanciamos el motor de conocimiento
motor = DiagnosticoMedico()
motor.reset()

# Agregamos hechos
motor.declare(Sintoma(nombre="fiebre"))
motor.declare(Sintoma(nombre="dolor de cabeza"))

# Ejecutamos las reglas
motor.run()


Podría ser migraña o estrés.
Podría tener una infección.


### Explicacion

---

```python
# Paso 1: Importar la librería y las clases 
from experta import Fact, Rule, KnowledgeEngine
```


```python
# Paso 2: Definir un hecho
class Sintoma(Fact):
    pass
```
Aquí defines un tipo de hecho que el sistema puede conocer, llamado Sintoma.
Hereda de Fact, que es una clase base de la librería.
Los hechos son como datos que el sistema usará para decidir.
```python
# Paso 3: Crear el sistema experto
class DiagnosticoMedico(KnowledgeEngine):
    
    @Rule(Sintoma(nombre="fiebre"))
    def enfermedad_fiebre(self):
        print("Podría tener una infección.")

    @Rule(Sintoma(nombre="dolor de cabeza"))
    def enfermedad_cabeza(self):
        print("Podría ser migraña o estrés.")
```
Creas una clase que representa tu motor de inferencia. Hereda de KnowledgeEngine, que gestiona reglas y hechos.

Dentro de esta clase defines reglas usando los decoradores:
```python
@Rule(Sintoma(nombre="fiebre"))
    def enfermedad_fiebre(self):
        print("Podría tener una infección.")

```
Aquí defines una regla:

Si hay un hecho Sintoma(nombre="fiebre"),

Entonces ejecuta la función enfermedad_fiebre.



```python
# Paso 4: Instanciar el motor
motor = DiagnosticoMedico()
motor.reset()
```
Crea una instancia de tu motor.

reset() borra cualquier hecho anterior y prepara el motor para nuevos hechos.

```python
# Paso 5: Declarar hechos
motor.declare(Sintoma(nombre="fiebre"))
motor.declare(Sintoma(nombre="dolor de cabeza"))
```
Aquí le dices al sistema:

“Tengo un paciente con fiebre y dolor de cabeza.”

Estos hechos son los datos de entrada para que el sistema tome decisiones.
```python
# Paso 6: Ejecutar el sistema
motor.run()
```

## Uso de Prioridades (salience)


si no hay prioridades entonces, el último hecho declarado que activa una regla será la primera en ejecutarse.

por eso en el ejemplo anterior lo primero que ejecuta es 

Podría ser migraña o estrés.


para controlar el orden de ejecución usamos **salience**

In [20]:
# 🔥 Sistema experto con prioridades usando `salience` )

# Paso 1: Definir los hechos
class Sintoma(Fact):
    pass

# Paso 2: Crear el motor de conocimiento con reglas y prioridades
class DiagnosticoMedico(KnowledgeEngine):

    # Esta regla tiene mayor prioridad con salience=10
    @Rule(Sintoma(nombre="fiebre"), salience=10)
    def enfermedad_fiebre(self):
        print("🔺 Podría tener una infección.")

    # Esta tiene menor prioridad
    @Rule(Sintoma(nombre="dolor de cabeza"), salience=5)
    def enfermedad_cabeza(self):
        print("🔻 Podría ser migraña o estrés.")

# Paso 3: Instanciar y preparar el motor
motor = DiagnosticoMedico()
motor.reset()

# Paso 4: Declarar hechos (en orden invertido para probar salience)
motor.declare(Sintoma(nombre="fiebre"))
motor.declare(Sintoma(nombre="dolor de cabeza"))

# Paso 5: Ejecutar el motor de inferencia
motor.run()


🔺 Podría tener una infección.
🔻 Podría ser migraña o estrés.


## Ejemplo 2 con uso de salience

In [26]:
from experta import Fact, Rule, KnowledgeEngine

# hechos
class Computadora(Fact):
    """Estado de una computadora."""
    pass

# creacion de reglas

class SoporteComputadora(KnowledgeEngine):
    @Rule(Computadora(estado='no_responde'), salience=1)
    def reiniciar_computadora(self):
        print("Reiniciando la computadora 1.")
        self.declare(Computadora(estado='funcionando'))

    # Esta regla se incluye a proposito para aplicar salience
    @Rule(Computadora(estado='no_responde'), salience=2)
    def reiniciar_computadora2(self):
        print("Reiniciando la computadora 2.")

    @Rule(Computadora(estado='funcionando'), salience=3)
    def computadora_ok(self):
        print("La computadora está funcionando correctamente.")

# Ejecución

engine = SoporteComputadora()
engine.reset()
engine.declare(Computadora(estado='no_responde'))
engine.run()

Reiniciando la computadora 2.
Reiniciando la computadora 1.
La computadora está funcionando correctamente.


## usando AND y OR

usar operadores lógicos como AND (con &) y OR (con |) en las reglas para combinar condiciones múltiples. Esto permite que una regla se active solo cuando se cumplen varios síntomas al mismo tiempo, o uno u otro.



In [1]:

from experta import Fact, Rule, KnowledgeEngine, AND, OR

# Paso 1: Definir el hecho
class Sintoma(Fact):
    pass

# Paso 2: Crear el sistema experto
class DiagnosticoMedico(KnowledgeEngine):

    # Regla con AND: fiebre Y dolor de cabeza
    @Rule(AND(Sintoma(nombre="fiebre"), Sintoma(nombre="dolor de cabeza")))
    def posible_gripa(self):
        print("🟡 Podría ser una gripe fuerte.")

    # Regla con OR: fiebre O tos
    @Rule(OR(Sintoma(nombre="fiebre"), Sintoma(nombre="tos")))
    def posible_infeccion(self):
        print("🟠 Puede haber una infección en curso.")

    # Reglas individuales (como antes)
    @Rule(Sintoma(nombre="fiebre"))
    def solo_fiebre(self):
        print("🔺 Solo fiebre detectada.")

    @Rule(Sintoma(nombre="dolor de cabeza"))
    def solo_cabeza(self):
        print("🔻 Solo dolor de cabeza detectado.")

# Paso 3: Instanciar el motor
motor = DiagnosticoMedico()
motor.reset()

# Paso 4: Declarar hechos
motor.declare(Sintoma(nombre="fiebre"))
motor.declare(Sintoma(nombre="dolor de cabeza"))

# Paso 5: Ejecutar el motor
motor.run()


AttributeError: module 'collections' has no attribute 'Mapping'

tambien esta el operador lógico NOT

### Ejemplo

In [ ]:
from experta import Fact, KnowledgeEngine, Rule, AND, OR, NOT

# se definen hechos

class Planta(Fact):
    """Información sobre la planta."""
    pass

class Ambiente(Fact):
    """Las condiciones ambientales que afectan la planta."""
    pass

# Se define el motor de conocimiento donde se implementan las reglas utilizando los operadores lógicos.

class AsistenteCuidadoPlantas(KnowledgeEngine):
    @Rule(AND(Planta(tipo='tomate', nivel_agua='bajo'), Ambiente(sol='alto', humedad='media')))
    def cuidado_tomate(self):
        print("Riega la planta de tomate más frecuentemente y revisala por quemaduras de sol.")

    @Rule(OR(Planta(tipo='pepino', nivel_agua='alto'), Ambiente(humedad='alta')))
    def cuidado_pepino(self):
        print("Reduce el riego del pepino y asegura buena circulación de aire.")

    @Rule(NOT(Planta(tipo='lechuga', salud='buena')))
    def cuidado_lechuga(self):
        print("Revisa la lechuga por plagas y enfermedades, considera usar pesticidas naturales.")

    # Agregando más reglas que combinan AND, OR, NOT
    @Rule(AND(Planta(tipo='zanahoria', etapa_crecimiento='plántula'),
              OR(Ambiente(temperatura='fría'), Ambiente(humedad='baja')),
              NOT(Ambiente(lluvia='reciente'))))
    def cuidado_zanahoria(self):
        print("Protege las plántulas de zanahoria del frío y mantenga la humedad del suelo sin excesos de riego.")

# ejecucion del sistema

engine = AsistenteCuidadoPlantas()
engine.reset()
engine.declare(Planta(tipo='tomate', nivel_agua='bajo'))
engine.declare(Ambiente(sol='alto', humedad='media'))
engine.declare(Planta(tipo='pepino', nivel_agua='alto'))
engine.declare(Ambiente(humedad='alta'))
engine.declare(Planta(tipo='lechuga', salud='pobre'))
engine.declare(Planta(tipo='zanahoria', etapa_crecimiento='plántula'))
engine.declare(Ambiente(temperatura='fría', humedad='baja', lluvia='ninguna'))
engine.run()

Protege las plántulas de zanahoria del frío y mantenga la humedad del suelo sin excesos de riego.
Protege las plántulas de zanahoria del frío y mantenga la humedad del suelo sin excesos de riego.
Reduce el riego del pepino y asegura buena circulación de aire.
Reduce el riego del pepino y asegura buena circulación de aire.
Riega la planta de tomate más frecuentemente y revisala por quemaduras de sol.
Revisa la lechuga por plagas y enfermedades, considera usar pesticidas naturales.


## Razonamiento con cambio de estados
Uso de funciones retract() y declare().

retract(): Esta función se utiliza para eliminar hechos de la memoria de trabajo del sistema experto.

declare(): se utiliza para definir dinámicamente nuevos hechos o clases de hechos durante la ejecución del programa

### Ejercicio
Se desea implementar un sistema experto donde un agente decide si regar o no las plantas según la humedad del suelo y el clima. Vamos a detallar el proceso paso a paso:

Se definen los hechos: En este caso, podríamos tener un hecho Clima que pueda ser soleado, nublado o lluvioso, y otro hecho HumedadSuelo que pueda ser bajo, medio o alto.

Definir las reglas: Las reglas podrían especificar que si la humedad del suelo es bajo y el clima no es lluvioso, entonces se debe regar las plantas. Si se decide regar las plantas, entonces la humedad del suelo debe cambiar a alto.

Modificar hechos en las acciones de las reglas: Cuando una regla se dispara, la acción podría modificar el hecho de HumedadSuelo para reflejar que las plantas han sido regadas.

Además de decidir si regar las plantas, el sistema también decide si es necesario aplicar fertilizante según el estado de las plantas y si es necesario emitir una alerta si las condiciones son adversas para el crecimiento de las plantas.

In [27]:
from experta import *

# Definir hechos

class Clima(Fact):
    """Info about the weather."""
    pass

class HumedadSuelo(Fact):
    """Info about the soil moisture."""
    pass

class EstadoPlantas(Fact):
    """Info about the plants' health."""
    pass

class Alerta(Fact):
    """Alert system for extreme conditions."""
    pass

# Reglas

class SistemaRiego(KnowledgeEngine):
    @Rule(OR(AND(Clima(tipo='soleado'), HumedadSuelo(nivel='bajo')),
             AND(Clima(tipo='nublado'), HumedadSuelo(nivel='bajo'))))
    def regar_plantas(self):
        print("Acción: Regar las plantas.")
        self.retract(self.fact_humedad) # Borra el hecho fact_humedad: ver cargar_datos_iniciales(self) (evita dos hechos del mismo tipo)
        self.fact_humedad = self.declare(HumedadSuelo(nivel='alto')) # se crea el nuevo hecho fact_humedad
        print("Nuevo estado: Humedad del suelo alta.")

    @Rule(AND(HumedadSuelo(nivel='alto'), NOT(EstadoPlantas(estado='saludable'))))
    def aplicar_fertilizante(self):
        print("Acción: Aplicar fertilizante.")
        self.declare(EstadoPlantas(estado='saludable'))
        print("Nuevo estado: Plantas saludables.")

    @Rule(AND(Clima(tipo='lluvioso'), HumedadSuelo(nivel='alto')))
    def emitir_alerta(self):
        print("Acción: Emitir alerta de riesgo de inundación.")
        self.declare(Alerta(tipo='inundación'))
        print("Alerta: Riesgo de inundación activada.")

    def __init__(self):
        super().__init__()
        self.fact_humedad = None


    def cargar_datos_iniciales(self):
        self.declare(Clima(tipo='soleado'))
        self.fact_humedad = self.declare(HumedadSuelo(nivel='bajo'))
        self.declare(EstadoPlantas(estado='no saludable'))

# Ejecución

sistema = SistemaRiego()
sistema.reset()  # Prepara el motor de inferencia
sistema.cargar_datos_iniciales()  # Cargamos los hechos iniciales
sistema.run()  # Ejecutar el motor de inferencia

Acción: Regar las plantas.
Nuevo estado: Humedad del suelo alta.
Acción: Aplicar fertilizante.
Nuevo estado: Plantas saludables.


Este sistema es dinámico y permite la interacción entre varias reglas que pueden modificar la base de hechos y, a su vez, desencadenar otras reglas, mostrando un ejemplo más complejo de cómo los sistemas basados en reglas pueden gestionar una serie de decisiones interdependientes.

## Explicacion

```python
@Rule(OR(
    AND(Clima(tipo='soleado'), HumedadSuelo(nivel='bajo')),
    AND(Clima(tipo='nublado'), HumedadSuelo(nivel='bajo'))
))
def regar_plantas(self):
    print("Acción: Regar las plantas.")
    self.retract(self.fact_humedad)
    self.fact_humedad = self.declare(HumedadSuelo(nivel='alto'))
    print("Nuevo estado: Humedad del suelo alta.")
```

Si el clima es soleado o nublado y la humedad del suelo es baja, se riega.

Luego, se actualiza el hecho de humedad a alto, usando retract() y declare() para evitar duplicados.




```python
    def __init__(self):
        super().__init__()
        self.fact_humedad = None
```

este seria el constructor de la clase SistemaRiego que hereda todo de KnowledgeEngine
y estamos añadiendo el atributo humedad en None